## TYTANチュートリアル（線形回帰）

2023年6月4日

ビネクラ安田

出典：[量子アニーリング（QUBO）で線形回帰（最小二乗法）](https://vigne-cla.com/21-28/)

### 問題
ビリリダマ10匹のデータがある。最小二乗法で回帰直線を求めよ。
```
height(m)	weight(kg)
0.31	5.75
0.4	8.56
0.47	8.42
0.4	7.78
0.54	10.25
0.36	6.79
0.56	11.51
0.43	7.66
0.32	6.99
0.6	10.61
```
<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/06/21-28_1.png" width = 45%>
</div>

### QUBOモデルでは何が設定できるか？（おさらい）

**<font color="red">量子ビットを8個（8bit）用いて0～255の整数を表す</font>**

例）xを0～255の整数で表す（x0～x7を使用）
```
x = 128*x0 + 64*x1 + 32*x2 + 16*x3 + 8*x4 + 4*x5 + 2*x6 + x7
```
これがチュートリアル27（連立方程式を解く）にあるので予習しておくと良い。

他、条件式のチートシートは → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 考え方

高さx、重さyとして「y=ax+b」の傾きaと切片bを求める。

参考に、Excelで回帰直線を求めたのがこちら。
<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/06/21-28_2.png" width = 45%>
</div>

最小二乗法では**「誤差の二乗の合計」**ができるだけ小さくなるような傾きaと切片bを求める。そこで、量子アニーリングではこの**「誤差の二乗の合計」**をエネルギーの**ペナルティ**に設定してやれば、最小エネルギーの解を求めることがそのまま最小二乗法に相当する。

例えば x=0.31 に着目すると、実データは 5.75 で、近似直線上の値は (a×0.31 + b) である。誤差は (5.75 – (a×0.31 + b)) で、これの二乗をペナルティにする。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/06/21-28_3.png" width = 40%>
</div>

ここで、aとbは量子ビットをそのまま割り当ててしまうと０と１しか取り得ない。そこで量子ビットを8bit使って0～255までの256段階の数を表す方法を活用する。この256段階を10～20に規格化することで**「10～20の間の256段階の数値」**を表せる。aは10～20の間と見積もってこのように表現する。

```
#a0～a7は量子ビット
a = 10 + 10 * ((128*a0 + 64*a1 + 32*a2 + 16*a3 + 8*a4 + 4*a5 + 2*a6 + a7) / 255)
```

同様に、0～1に規格化することで**「0～1の間の256段階の数値」**を表せる。bは0～1の間と見積もってこのようにする。

```
#b0～b7は量子ビット
b = 0 + 1 * ((128*b0 + 64*b1 + 32*b2 + 16*b3 + 8*b4 + 4*b5 + 2*b6 + b7) / 255)
```

誤差の二乗ペナルティはこうなる。

```
H += (5.75 - (a*0.31 + b))**2
```

このaとbはそれぞれ8個の量子ビットの集合体で、展開した式はかなり長くなる。「こんな式立てて大丈夫なの？」と心配になるが大丈夫。QUBOでは2個の量子ビットの掛け算まで設定可能で、3個以上の量子ビットの掛け算はあってはいけない。冷静に展開してみると3個以上の掛け算は現れないので大丈夫。

## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [2]:
from tytan import symbols, Compile, sampler
import numpy as np

#量子ビットを用意する
a0 = symbols('a0')
a1 = symbols('a1')
a2 = symbols('a2')
a3 = symbols('a3')
a4 = symbols('a4')
a5 = symbols('a5')
a6 = symbols('a6')
a7 = symbols('a7')

b0 = symbols('b0')
b1 = symbols('b1')
b2 = symbols('b2')
b3 = symbols('b3')
b4 = symbols('b4')
b5 = symbols('b5')
b6 = symbols('b6')
b7 = symbols('b7')

#aを2進数（8bit）で表す、10～20で規格化
a = 10 + 10 * ((128*a0 + 64*a1 + 32*a2 + 16*a3 + 8*a4 + 4*a5 + 2*a6 + a7) / 255)

#bを2進数（8bit）で表す、0～1で規格化
b = 0 + 1 * ((128*b0 + 64*b1 + 32*b2 + 16*b3 + 8*b4 + 4*b5 + 2*b6 + b7) / 255)

#データ1の誤差二乗ペナルティ
H = 0
H += (5.75 - (a*0.31 + b))**2

#以降同様
H += (8.56 - (a*0.4 + b))**2
H += (8.42 - (a*0.47 + b))**2
H += (7.78 - (a*0.4 + b))**2
H += (10.25 - (a*0.54 + b))**2
H += (6.79 - (a*0.36 + b))**2
H += (11.51 - (a*0.56 + b))**2
H += (7.66 - (a*0.43 + b))**2
H += (6.99 - (a*0.32 + b))**2
H += (10.61 - (a*0.6 + b))**2


#コンパイル
qubo, offset = Compile(H).get_qubo()

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=500)

#上位5件
for r in result[:5]:
    print(r)
    #2進数から戻して確認
    a0,a1,a2,a3,a4,a5,a6,a7 = list(r[0].values())[:8]
    b0,b1,b2,b3,b4,b5,b6,b7 = list(r[0].values())[8:]
    print(f'a = {10 + 10 * ((128*a0 + 64*a1 + 32*a2 + 16*a3 + 8*a4 + 4*a5 + 2*a6 + a7) / 255)}')
    print(f'b = {0 + 1 * ((128*b0 + 64*b1 + 32*b2 + 16*b3 + 8*b4 + 4*b5 + 2*b6 + b7) / 255)}')

[{'a0': 1.0, 'a1': 0.0, 'a2': 1.0, 'a3': 1.0, 'a4': 1.0, 'a5': 0.0, 'a6': 0.0, 'a7': 0.0, 'b0': 1.0, 'b1': 1.0, 'b2': 0.0, 'b3': 1.0, 'b4': 1.0, 'b5': 1.0, 'b6': 1.0, 'b7': 1.0}, -168.14858346789703, 182]
a = 17.215686274509803
b = 0.8745098039215686
[{'a0': 1.0, 'a1': 0.0, 'a2': 1.0, 'a3': 1.0, 'a4': 0.0, 'a5': 1.0, 'a6': 1.0, 'a7': 1.0, 'b0': 1.0, 'b1': 1.0, 'b2': 1.0, 'b3': 0.0, 'b4': 0.0, 'b5': 0.0, 'b6': 1.0, 'b7': 1.0}, -168.1484976547482, 38]
a = 17.176470588235293
b = 0.8901960784313725
[{'a0': 1.0, 'a1': 0.0, 'a2': 1.0, 'a3': 1.0, 'a4': 0.0, 'a5': 1.0, 'a6': 1.0, 'a7': 1.0, 'b0': 1.0, 'b1': 1.0, 'b2': 1.0, 'b3': 0.0, 'b4': 0.0, 'b5': 1.0, 'b6': 0.0, 'b7': 0.0}, -168.1484484429066, 43]
a = 17.176470588235293
b = 0.8941176470588236
[{'a0': 1.0, 'a1': 0.0, 'a2': 1.0, 'a3': 1.0, 'a4': 1.0, 'a5': 0.0, 'a6': 0.0, 'a7': 0.0, 'b0': 1.0, 'b1': 1.0, 'b2': 1.0, 'b3': 0.0, 'b4': 0.0, 'b5': 0.0, 'b6': 0.0, 'b7': 0.0}, -168.14841430219153, 15]
a = 17.215686274509803
b = 0.8784313725490196
[

最良の解は (a, b) = (17.216, 0.875) となった。Excelで求めた傾きと切片は (17,206, 0.879) だったためかなり近い値が得られたことになる。

aとbはそれぞれ8bitで表現したため256段階の飛び飛びの値しか取り得ない。したがって**厳密な最適パラメータは求まらない。**より厳密に近いパラメータを求めたければ、aは17～18の範囲に絞って、bは0.8～0.9の範囲に絞ってさらにアニーリングを繰り返すと良い。または、9bitで表現すれば512段階で刻めるのでそういうアプローチでも良い。

余談で、「ビリリダマは大きくなるほど密度が下がる」に興味があれば

→[ポケモンの高さと重さを調べてみたら何かおかしかった](https://vigne-cla.com/16-16/)